In [1]:
import sys
{sys.executable}

{'/opt/conda/bin/python'}

In [2]:
import torch
torch.__version__

'1.4.0'

In [3]:
# Imports for this tutorial
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import json
from collections import defaultdict, OrderedDict
import math
import numpy as np

torch.manual_seed(1)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
f  = open("test.txt", "r") 

In [144]:
from itertools import chain
data_test = []
tag_test = []
t = []
sent = []
label = []
vocab = {}
tag_map = {}
for i, line in enumerate(f1):  
    if line.split(): #on ne prend pas en compte les listes vides
        vocab[line.split()[0]] = i
        sent.append(line.split()[0])
        
        tag = [s for s in line.split()[1:] if s not in string.punctuation]
        tag_test.append(tag[-1])
        t.append(tag[-1])
        
    else:
        if tag_test:
            label.append(" ".join(tag_test))
            tag_test = []
        if sent:
            data_test.append(" ".join(sent))
            sent = []
        
for j, w in enumerate(t):
    tag_map[w] = j       

In [145]:
maxi = max(vocab.values())

In [158]:
vocab.update(UNK = maxi+1, PAD = maxi+2)

In [124]:
train_sentences = []        
train_labels = []

for sentence in data_test:
    s = [vocab[token] if token in vocab 
        else vocab['UNK']
        for token in sentence.split(' ')]
    train_sentences.append(s)

for sentence in label:
    #replace each label by its index
    l = [tag_map[la] for la in sentence.split(' ')]
    train_labels.append(l)  

In [228]:
# utilisation du GPU si possible 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 64
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device, sort = False)

In [197]:
from torch.autograd import Variable

In [286]:
t =[ [[50068, 50069, 732, 4, 5, 49476, 50345, 8, 49392, 41417, 11, 50347],
  [14, 15],
  [1331, 50345, 50311, 48188, 48189, 49192],
     [12,84,6,4545,2111,5,8,9,45]],[[50068, 50069, 732, 4, 5, 49476, 50345, 8, 49392, 41417, 11, 50347], [14, 15]] ]
print(t[1])

In [394]:
train_iterator = data_iterator(train_sentences, train_labels, batch_size = 64)    


In [458]:
for i,j in train_iterator:
    print(i)

ValueError: too many values to unpack (expected 2)

In [334]:
x = torch.rand((71, 32, 1))
# x.shape = torch.Size([71, 32, 1])
px = torch.cat((torch.zeros(29, 32, 1, dtype=x.dtype, device=x.device), x), dim=0)
# px.shape = torch.Size([100, 32, 1])

In [351]:
n = 64 #batch size  
# using list comprehension  
batch_sentences = [train_sentences[i:i + n] for i in range(0, len(train_sentences), n)]
del batch_sentences[-1]

In [481]:
a, b = data_iterator(train_sentences, train_labels, batch_size = 64)    

torch.Size([72, 64])
torch.Size([18, 64])
torch.Size([33, 64])
torch.Size([124, 64])
torch.Size([36, 64])


In [480]:
def data_iterator(train_sentences, train_labels, batch_size = 64):
    n = batch_size
    # création des batchs
    batch_sentences = [train_sentences[i:i + n] for i in range(0, len(train_sentences), n)]
    del batch_sentences[-1] # on supprime le dernier batch car il est pas de longueur 64
    
    batch_tags = [train_labels[i:i + n] for i in range(0, len(train_labels), n)]    
    del batch_tags[-1] 
    
    batchs_data = []
    batchs_labels = [] 
    
    for i in range(len(batch_sentences)):
        #compute length of longest sentence in batch
        batch_max_len = max([len(s) for s in batch_sentences[i]])
        #prepare a numpy array with the data, initializing the data with 'PAD' 
        #and all labels with -1; initializing labels to -1 differentiates tokens 
        #with tags from 'PAD' tokens
        batch_data = vocab["PAD"]*np.ones((len(batch_sentences[i]), batch_max_len))
        batch_labels = -1*np.ones((len(batch_sentences[i]), batch_max_len))
        #copy the data to the numpy array
        for j in range(len(batch_sentences[i])):
            cur_len = len(batch_sentences[i][j])
            batch_data[j][:cur_len] = batch_sentences[i][j]
            batch_labels[j][:cur_len] = batch_tags[i][j]
            
        batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels) 
        batch_data = batch_data.transpose(0, 1)
        batch_labels = batch_labels.transpose(0, 1)
        batchs_data.append(batch_data.squeeze(1))
        batchs_labels.append(batch_labels.squeeze(1))
        
    
    """
    batchs_data = torch.cat(batchs_data,dim = 0)  
    batchs_labels = torch.cat(batchs_labels, dim = 0)  
    #since all data are indices, we convert them to torch LongTensors
    batchs_data, batchs_labels = torch.LongTensor(batchs_data), torch.LongTensor(batchs_labels)
    #convert Tensors to Variables
    batchs_data, batchs_labels = Variable(batchs_data), Variable(batchs_labels)
    print(batchs_data.shape)
    print('l',batchs_labels.shape)
    """
    return batchs_data, batchs_labels

In [484]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim,bidirectional):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim, bidirectional=bidirectional)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
    def forward(self, text):

        embedded = self.embedding(text)
               
        output, hidden = self.rnn(embedded)
        
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
        
        return self.fc(hidden)


In [485]:
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(tag_map)
BIDIRECTIONAL = True
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,BIDIRECTIONAL )

In [486]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [488]:
n = 64
# création des batchs
batch_sentences = [train_sentences[i:i + n] for i in range(0, len(train_sentences), n)]
del batch_sentences[-1] # on supprime le dernier batch car il est pas de longueur 64
    
batch_tags = [train_labels[i:i + n] for i in range(0, len(train_labels), n)]    
del batch_tags[-1] 

epoch_loss = 0
epoch_acc = 0
model.train()
        
for i in range(len(batch_sentences)):
    #compute length of longest sentence in batch
    batch_max_len = max([len(s) for s in batch_sentences[i]])
    #prepare a numpy array with the data, initializing the data with 'PAD' 
    #and all labels with -1; initializing labels to -1 differentiates tokens 
    #with tags from 'PAD' tokens
    batch_data = vocab["PAD"]*np.ones((len(batch_sentences[i]), batch_max_len))
    batch_labels = -1*np.ones((len(batch_sentences[i]), batch_max_len))
    #copy the data to the numpy array
    for j in range(len(batch_sentences[i])):
        cur_len = len(batch_sentences[i][j])
        batch_data[j][:cur_len] = batch_sentences[i][j]
        batch_labels[j][:cur_len] = batch_tags[i][j]
            
    batch_data, batch_labels = torch.LongTensor(batch_data), torch.LongTensor(batch_labels) 
    batch_data = batch_data.transpose(0, 1)
    batch_labels = batch_labels.transpose(0, 1)

    optimizer.zero_grad()    
    text = batch_data.to(device)
    predictions = model(text).squeeze(1)    
    loss = criterion(predictions, batch_labels)    
    acc = binary_accuracy(predictions, batch_labels)    
    loss.backward()
        
    optimizer.step()
        
    epoch_loss += loss.item()
    epoch_acc += acc.item()

ValueError: Target size (torch.Size([72, 64])) must be the same as input size (torch.Size([64]))

In [234]:
def binary_accuracy(preds, y):
    """
    Retourne l'accuracy par batch
    """
    #arrondi la prédiction à l'entier le plus proche
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: -7132.589 | Train Acc: 4.52%
	 Val. Loss: -12209.397 |  Val. Acc: 4.78%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: -16873.896 | Train Acc: 4.55%
	 Val. Loss: -21499.249 |  Val. Acc: 4.78%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: -26063.534 | Train Acc: 4.52%
	 Val. Loss: -30691.432 |  Val. Acc: 4.78%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: -35041.484 | Train Acc: 4.55%
	 Val. Loss: -39791.703 |  Val. Acc: 4.78%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: -44442.868 | Train Acc: 4.52%
	 Val. Loss: -48840.225 |  Val. Acc: 4.78%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: -53533.904 | Train Acc: 4.50%
	 Val. Loss: -57877.539 |  Val. Acc: 4.78%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: -62506.677 | Train Acc: 4.50%
	 Val. Loss: -66997.074 |  Val. Acc: 4.78%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: -71536.145 | Train Acc: 4.57%
	 Val. Loss: -75940.350 |  Val. Acc: 4.78%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: -80561.081 | Train Acc: 4.50%


In [211]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self,input_dim, embedding_dim, hidden_dim, output_dim,bidirectional):
        super(Net, self).__init__()

    #maps each token to an embedding_dim vector
    self.embedding = nn.Embedding(params.vocab_size, params.embedding_dim)

    #the LSTM takens embedded sentence
    self.lstm = nn.LSTM(params.embedding_dim, params.lstm_hidden_dim, batch_first=True)

    #fc layer transforms the output to give the final output layer
    self.fc = nn.Linear(params.lstm_hidden_dim, params.number_of_tags)
    
    def forward(self, s):
        #apply the embedding layer that maps each token to its embedding
        s = self.embedding(s)   # dim: batch_size x batch_max_len x embedding_dim

        #run the LSTM along the sentences of length batch_max_len
        s, _ = self.lstm(s)     # dim: batch_size x batch_max_len x lstm_hidden_dim                

        #reshape the Variable so that each row contains one token
        s = s.view(-1, s.shape[2])  # dim: batch_size*batch_max_len x lstm_hidden_dim

        #apply the fully connected layer and obtain the output for each token
        s = self.fc(s)          # dim: batch_size*batch_max_len x num_tags

        return F.log_softmax(s, dim=1)   # dim: batch_size*batch_max_len x num_tags

AttributeError: type object 'params' has no attribute 'vocab_size'

In [125]:
len(train_sentences)

3682

In [126]:
len(train_labels)

3682

In [128]:
import pandas as pd
#train = pd.DataFrame({'text': train, 'label': label_train})
test = pd.DataFrame({'text': train_sentences, 'label': train_labels})
#valid = pd.DataFrame({'text': valid, 'label': label_valid})

#train.to_csv (r'data/train.csv', index = False, header=True)
test.to_csv (r'data_ner/test.csv', index = False, header=True)
#valid.to_csv (r'data/valid.csv', index = False, header=True)

In [133]:
import torch
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#def generate_bigrams(x):
#    n_grams = set(zip(*[x[i:] for i in range(2)]))
#    for n_gram in n_grams:
#        x.append(' '.join(n_gram))
#    return x

TEXT = data.Field(sequential=True,lower=True, tokenize = 'spacy', #preprocessing = generate_bigrams, 
                  include_lengths=True)
LABEL = data.LabelField(dtype = torch.float)
train_data, valid_data, test_data = data.TabularDataset.splits(
        path='./data_ner/', train='test.csv',
        validation='test.csv', test='test.csv', format='csv', skip_header=True,
        fields=[('text', TEXT), ('label', LABEL)])

print(f'Taille des données train: {len(train_data)}')
print(f'Taille des données de validation: {len(valid_data)}')
print(f'Taille des données test: {len(test_data)}')


Taille des données train: 3682
Taille des données de validation: 3682
Taille des données test: 3682


In [134]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [135]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device,sort_key=lambda x: len(x.text),
    sort_within_batch = True)